# Bash

Когда речь заходит об обработке больших данных, на первый план выходят задачи по грамотной манипуляции данными, конвертированию и выгрузке, настройке специализированных инструментов и так далее. Чтобы уметь решать такие задачи с кайфом, нужно обрести уверенные навыки работы в терминале и командных оболочках.

Bash - наиболее популярная командная оболочка в UNIX-like система. Когда вы пользуетесь терминалом на своем компьютере, то вы почти наверное используете именно Bash.

В баш встроено огромное количество функциональности, которая не ограничивается лишь запуском программ.

# Потоки данных

Сильная сторона языка Bash - это возможности по работе с перенаправлениями потоками данных между подпрограммами.

Для Bash каждую программу можно представить в следующем виде - это черный ящик, которому

На вход подается какой-то источник данных, а также аргументы запуска

На выход выдает два источника данных - это вывод результата и вывод ошибок.

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/bash-flow-control-1.png">

Чаще всего к такому "ящику" подключаются три стандартный источника данных - это stdin, stdout и stderr.

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/bash-flow-control-2.png">

Все эти три источника, как и все в UNIX, являются виртуальными файлами и выполняют следующие функции

* stdin представляет весь поток данных, который пользователь вводит с клавиатуры
* stdout представляет весь поток данных, который программа печатает на экран
* stderr представляет весь поток данных об ошибках в работе программы, который программа также печатает на экран

Все эти источники находятся по следующим путям: `/dev/stdout`, `/dev/stdin`, `/dev/stderr`. Можно заглянуть в директорию `/dev/` и посмотреть, сколько еще виртуальных источников данных есть в компьютере.

In [ ]:
! mkdir -p tempsem2

In [ ]:
%cd tempsem2

In [ ]:
! ls /dev

Волшебство UNIX заключается в том, что мы можем работать с этими источниками как с самыми обычными файлами. Например, давайте попробуем что-то написать в файл /dev/stdout

In [ ]:
%%writefile hello-stdout.py
with open('/dev/stdout', 'w') as f:
    f.write("HELLO, STDOUT!")  # Пишем в специальный файл, вместо print

In [ ]:
! python3 hello-stdout.py

Вместо стандартных потоков ввода\вывода можно подставлять произвольные источники данных.

**Оператор >** позволяет перенаправлять стандартный вывод в любой другой файл. Попробуем, например, написать что-то в новый файл используя команду echo.

In [ ]:
! echo "dsjfhkdsfh"

In [ ]:
! echo "message from echo" > file.txt

In [ ]:
! cat file.txt

In [ ]:
! cat file.txt file.txt # Читает файл два раза

In [ ]:
! cat file.txt file.txt > doubled-file.txt

In [ ]:
! cat doubled-file.txt

Если файл уже существует, то этот оператор полностью перезатрет его содержимое

In [ ]:
! echo "new message" > doubled-file.txt

In [ ]:
! cat doubled-file.txt

**Оператор >>** позволяет не перезаписывать целиком файл, а лишь добавить в конец новые данные

In [19]:
! cat file.txt file.txt > doubled-file.txt

In [20]:
! cat doubled-file.txt

message from echo
message from echo


In [21]:
! echo "new message" >> doubled-file.txt

In [22]:
! cat doubled-file.txt

message from echo
message from echo
new message


Помимо вывода программы, можно поменять и ее ввод.

**Оператор <** позволяет поменить стандартный ввод программы на другой файл, делая вид для программы, будто бы пользователь ввел эти данные с клавиатуры.

Напишем простенькую программу, которая читает пользовательский ввод с клавиатуры и потестируем этот оператор.

In [23]:
%%writefile repeat.py

data = input()
for i in range(3):
    print(data)

Writing repeat.py


In [24]:
! cat file.txt

message from echo


In [25]:
! python3 repeat.py < file.txt

message from echo
message from echo
message from echo


In [26]:
! python3 -c "import sys; print(list(sys.stdin))" < doubled-file.txt

['message from echo\n', 'message from echo\n', 'new message\n']


Если хочется направить на stdin несколько строк, но при этом не из файла, а прямо из скрипта, то можно воспользваться **оператором <<**. Для него нужно отдельно указать маркер начала и конца данных.

In [30]:
! python3 -c "print(2 ** 3)"

8


In [28]:
%%bash

python3 -c "import sys; print(list(sys.stdin))" <<BUBA
big
multiline
message
from
script
BUBA

['big\n', 'multiline\n', 'message\n', 'from\n', 'script\n']


Если хочется направить ровно одну строку, то можно воспользоваться **оператором <<<**. Он подаст на stdin ту строку, которую мы передадим в качестве аргумента.

In [29]:
%%bash

python3 -c "import sys; print(list(sys.stdin))" <<< "one line message from script"

['one line message from script\n']


Комбинирование этих возможностей может позволить решить некоторые задачи. Например генерировать какой-то файл на лету.

In [46]:
%%bash

cat > message2.txt <<BIBA 
This file was created directly from script
Using amazing features of Bash
BIBA

In [47]:
! cat message2.txt

This file was created directly from script
Using amazing features of Bash


In [32]:
! cat message.txt

This file was created directly from script
Using amazing features of Bash


Или например решать проблему с интерактивными программами, которые хотят какого-то действия от пользователя.

In [33]:
%%writefile interactive.py

answer = input("Are you sure you want to do X? (y/n)")
if answer == 'y':
    print("DOING X")
else:
    print("Canceling")

Writing interactive.py


In [34]:
! python3 interactive.py <<< "y"

Are you sure you want to do X? (y/n)DOING X


In [35]:
! python3 interactive.py <<< "n"

Are you sure you want to do X? (y/n)Canceling


Возможности перенаправлений не ограничиваются только статическими текстовыми файлами. 

**Оператор <()** позворяет представить вывод программы как специальный файл (чаще всего он выглядит как `/dev/fd/63`) . Основная особенность этого оператора от обычного **оператора >** в том, что не используется дополнительное место на диске. Читая из этого специального файла мы напрямую получаем доступ к выводу программы без необходимости сохранять его на диске в явном виде.

Сам оператор при применении запускает переданную программу, создает специальный файл и возращает путь до этого файла.

Этим можно воспользоваться, чтобы компоновать работу нескольких программ.

In [ ]:
! echo "biba" > file
! cat file

In [57]:
! echo <(echo PIPA)  # Напечатали путь до файла, в коротый подключен вывод программы echo PIPA

/dev/fd/63


In [51]:
! cat <(echo PIPA)

PIPA


In [59]:
%%writefile read-file.py

import sys

print("Path to file = {}".format(sys.argv[1]))

with open(sys.argv[1], 'r') as f:
    content = f.read()
    print(content)

Writing read-file.py


In [60]:
! cat file.txt

message from echo


In [61]:
! python3 read-file.py file.txt

Path to file = file.txt
message from echo



In [62]:
! python3 read-file.py <(echo PIPA)

Path to file = /dev/fd/63
PIPA



Комбинируя возможности операторов **<** и **<()** можно перенаправлять вывод одной команды в другой.

Наглядный пример - команда `yes`. Она решает уже рассмотренную проблему работы с интерактивными программами, без конца печатая символ `y` на стандартный вывод. 

(Интересный факт - люди любят соревноваться в "производительности" программы `yes`. Согласно [треду на Реддите](https://www.reddit.com/r/unix/comments/6gxduc/how_is_gnu_yes_so_fast/), рекорд - вывод `y` со скоростью 123 Гигабита в секунду. Зачем нужная такая производительность в команде `yes`? Ну чтобы было смешно.)

In [63]:
! python3 interactive.py < <(yes)

Are you sure you want to do X? (y/n)DOING X


Другой полезный пример - сравнение двух каталогов. Сущестует программа `diff`, которая получает два файла и выводит построчные различия в этий файлах. Можно скомбинировать ее с командой `ls`, чтобы научиться сравнивать директории.

In [64]:
! mkdir -p folder1
! touch folder1/file1.txt folder1/file2.txt folder1/file3.txt

! mkdir -p folder2
! touch folder2/file2.txt folder2/file3.txt folder2/file4.txt

In [65]:
! ls folder1

file1.txt  file2.txt  file3.txt


In [66]:
! ls folder2

file2.txt  file3.txt  file4.txt


In [67]:
! diff <(ls folder1) <(ls folder2)

1d0
< file1.txt
3a3
> file4.txt


Идея соединять процессы через stdin\stdout очень популярна и для этого есть более удобный интерфейс - pipes или **оператор |**

Все команды, соединенные через | запускаются одновременно и общаются друг с другом через stdin\stdout

In [68]:
! echo hello | cat

hello


In [69]:
! echo hi | wc -l  # wc считает сколько строк в входных данных

1


In [70]:
! cat doubled-file.txt | wc -l

3


In [71]:
! ls -l | wc -l

11


In [72]:
! ls -l | cat | wc -l | python3 -c "print(int(input()) * 2)"

22


In [73]:
! yes | python3 interactive.py

Are you sure you want to do X? (y/n)DOING X
yes: standard output: Broken pipe


In [74]:
! yes | head  # Смотрим первые 10 строк файла

y
y
y
y
y
y
y
y
y
y
yes: standard output: Broken pipe


Существует также родственный оператор **$()**. Он также запускает переданную программу, однако перенаправляет вывод не в файл, а прямо в bash. То есть вывод программы можно использовать как строку внутри скрипта.

In [75]:
! pwd

/home/ubuntu/tempsem2


In [76]:
! echo I am here - $(pwd)

I am here - /home/ubuntu/tempsem2


In [78]:
%%writefile file-to-read.txt
/etc/hosts

Writing file-to-read.txt


In [79]:
! cat $(cat file-to-read.txt)

# Your system has configured 'manage_etc_hosts' as True.
# As a result, if you wish for changes to this file to persist
# then you will need to either
# a.) make changes to the master file in /etc/cloud/templates/hosts.debian.tmpl
# b.) change or remove the value of 'manage_etc_hosts' in
#     /etc/cloud/cloud.cfg or cloud-config from user-data
#
127.0.1.1		epd233ve3f389jafokjp.auto.internal epd233ve3f389jafokjp
127.0.0.1		localhost
# The following lines are desirable for IPv6 capable hosts
::1		ip6-localhost ip6-loopback
ff02::1		ip6-allnodes
ff02::2		ip6-allrouters
169.254.169.254		metadata.internal
10.128.0.25		rc1a-dataproc-m-ymnek55ntqs3neie.mdb.yandexcloud.net


# Полезные программы

Bash - всего лишь оболочка и не умеет самостоятельно решать какие-то задачи. Основную работу выполняют установленные программы, которые можно вызывать из bash. Их можно использовать гораздо эффективнее, используя их вместе с возможностями оболочки.

### Head
head читает определенное количество данных с начала файла. Это полезно, например, когда хочется посмотреть на часть данных, которая лежит на диске.

In [80]:
! head /etc/hosts

# Your system has configured 'manage_etc_hosts' as True.
# As a result, if you wish for changes to this file to persist
# then you will need to either
# a.) make changes to the master file in /etc/cloud/templates/hosts.debian.tmpl
# b.) change or remove the value of 'manage_etc_hosts' in
#     /etc/cloud/cloud.cfg or cloud-config from user-data
#
127.0.1.1		epd233ve3f389jafokjp.auto.internal epd233ve3f389jafokjp
127.0.0.1		localhost
# The following lines are desirable for IPv6 capable hosts


In [81]:
! head -n 2 /etc/hosts  # Читаем только первые 2 строки

# Your system has configured 'manage_etc_hosts' as True.
# As a result, if you wish for changes to this file to persist


In [82]:
! head -c 10 /etc/hosts  # Читаем только первые 10 байт

# Your sys

In [83]:
! cat /etc/hosts | head -n 2  # Как и почти все программы, которые мы рассмотрим, умеет работать с вводом

# Your system has configured 'manage_etc_hosts' as True.
# As a result, if you wish for changes to this file to persist


### Tail
tail делает то же самое, что и head, но с конца файла

In [84]:
! tail /etc/hosts

#     /etc/cloud/cloud.cfg or cloud-config from user-data
#
127.0.1.1		epd233ve3f389jafokjp.auto.internal epd233ve3f389jafokjp
127.0.0.1		localhost
# The following lines are desirable for IPv6 capable hosts
::1		ip6-localhost ip6-loopback
ff02::1		ip6-allnodes
ff02::2		ip6-allrouters
169.254.169.254		metadata.internal
10.128.0.25		rc1a-dataproc-m-ymnek55ntqs3neie.mdb.yandexcloud.net


In [85]:
! tail -n 2 /etc/hosts

169.254.169.254		metadata.internal
10.128.0.25		rc1a-dataproc-m-ymnek55ntqs3neie.mdb.yandexcloud.net


In [87]:
! tail -n +3 /etc/hosts  # все строки после второй строки (включая вторую строку)

# then you will need to either
# a.) make changes to the master file in /etc/cloud/templates/hosts.debian.tmpl
# b.) change or remove the value of 'manage_etc_hosts' in
#     /etc/cloud/cloud.cfg or cloud-config from user-data
#
127.0.1.1		epd233ve3f389jafokjp.auto.internal epd233ve3f389jafokjp
127.0.0.1		localhost
# The following lines are desirable for IPv6 capable hosts
::1		ip6-localhost ip6-loopback
ff02::1		ip6-allnodes
ff02::2		ip6-allrouters
169.254.169.254		metadata.internal
10.128.0.25		rc1a-dataproc-m-ymnek55ntqs3neie.mdb.yandexcloud.net


In [88]:
! cat /etc/hosts | tail -n 2

169.254.169.254		metadata.internal
10.128.0.25		rc1a-dataproc-m-ymnek55ntqs3neie.mdb.yandexcloud.net


### Sort
sort сортирует входные данные. По умолчанию используется лексикографический порядок, но это поведение можно поменять с использованием специальных опций.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=sort&category=1

In [89]:
%%writefile numbers.txt
3
5
1
2
6
5
9
4
5
6
7
3
2
10
5
6

Writing numbers.txt


In [90]:
! sort numbers.txt  # Сортируем как строки

1
10
2
2
3
3
4
5
5
5
5
6
6
6
7
9


In [91]:
! cat numbers.txt | sort

1
10
2
2
3
3
4
5
5
5
5
6
6
6
7
9


In [92]:
! cat numbers.txt | sort -n # Сортируем как числа

1
2
2
3
3
4
5
5
5
5
6
6
6
7
9
10


Если в данных есть сразу несколько "колонок" (например в каждой строке есть значения, разделенные пробельным символом), то можно отдельно указать, по какому полю необходимо сортировать

In [93]:
%%writefile number-table.txt
1 10
2 9
3 8
4 7
5 6
6 5
7 4
8 3
9 2
10 1

Writing number-table.txt


In [94]:
! cat number-table.txt | sort -k1,1 -n

1 10
2 9
3 8
4 7
5 6
6 5
7 4
8 3
9 2
10 1


In [95]:
! cat number-table.txt | sort -k2,2 -n

10 1
9 2
8 3
7 4
6 5
5 6
4 7
3 8
2 9
1 10


In [96]:
! cat number-table.txt | sort -k2,2 -k1,1 -n

10 1
9 2
8 3
7 4
6 5
5 6
4 7
3 8
2 9
1 10


In [97]:
! cat numbers.txt | sort -n -r  # Сортируем в обратном порядке

10
9
7
6
6
6
5
5
5
5
4
3
3
2
2
1


### Shuf
shuf напротив, случайным образом перемешивает входящие данные

In [100]:
! cat numbers.txt | shuf

3
6
6
5
3
2
6
5
4
9
5
1
5
2
10
7


In [101]:
! cat numbers.txt | shuf

7
9
5
10
4
5
5
6
2
3
6
3
1
2
6
5


### Uniq
uniq оставляет только уникальные значения. Однако он корректно работает только с отсортированными данными. Для этого мы можем предварительно использовать sort.

Помимо операции схлопывания одинаковых значений, uniq также умеет считать простые статистики для схлопнувшихся групп. Этот функционал чем-то напоминает group by. Так, ключ -c считает количество элементов в каждой группе.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=uniq&category=1&russian=0

In [102]:
! cat numbers.txt | uniq  # Не совсем тот результат, что мы ожидаем

3
5
1
2
6
5
9
4
5
6
7
3
2
10
5
6


In [103]:
! cat numbers.txt | sort | uniq  # А вот так уже работает

1
10
2
3
4
5
6
7
9


По ходу "схлопывания" uniq умеет еще и подсчитывать количество схлопнутых элементов. Таким образом можно считать количество каждого элемента.

In [105]:
! cat numbers.txt | sort | uniq -c

      1 1
      1 10
      2 2
      2 3
      1 4
      4 5
      3 6
      1 7
      1 9


In [106]:
%%writefile words.txt
Lorem
ipsum
dolor
sit
met
consectetur
incididunt
elit
seddo
ipsum
tempor
incididunt
ut
laboret
dolor
ipsum
aliqua
ipsum

Writing words.txt


In [107]:
! cat words.txt | sort | uniq -c 

      1 aliqua
      1 consectetur
      2 dolor
      1 elit
      2 incididunt
      4 ipsum
      1 laboret
      1 Lorem
      1 met
      1 seddo
      1 sit
      1 tempor
      1 ut


### Wc
wc (word count) считает количество элементов во входных данных. По умолчанию считает три характеристики - количество строк, количество слов, количество байт. Различные опции позволяют считать какую-то одну из характеристик. Например -l считает количество строк в данных.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=wc&category=1&russian=0

In [108]:
! cat numbers.txt | wc

     16      16      33


In [109]:
! cat number-table.txt | wc

     10      20      42


In [110]:
! cat numbers.txt | wc -l # Количество элементов в файле

16


In [111]:
! cat numbers.txt | sort | uniq | wc -l  # Количество уникальных элементов в файле

9


### Cut

cut парсит строки, которые состоят из значений с разделителем. С помощью утилиты можно обрабатывать различные регулярные форматы данных, базирующиеся на разделителях. Например csv или tsv.

In [112]:
! wget https://raw.githubusercontent.com/ADKosm/lsml-2022-public/main/data/2/countries.csv

--2023-01-17 16:18:04--  https://raw.githubusercontent.com/ADKosm/lsml-2022-public/main/data/2/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14641 (14K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]  14.30K  --.-KB/s    in 0.001s  

2023-01-17 16:18:05 (12.6 MB/s) - ‘countries.csv’ saved [14641/14641]



In [113]:
! cat countries.csv

Country,Region,Population,Area (sq. mi.),GDP ($ per capita)
Afghanistan ,ASIA (EX. NEAR EAST)         ,31056997,647500,700.0
Albania ,EASTERN EUROPE                     ,3581655,28748,4500.0
Algeria ,NORTHERN AFRICA                    ,32930091,2381740,6000.0
American Samoa ,OCEANIA                            ,57794,199,8000.0
Andorra ,WESTERN EUROPE                     ,71201,468,19000.0
Angola ,SUB-SAHARAN AFRICA                 ,12127071,1246700,1900.0
Anguilla ,LATIN AMER. & CARIB    ,13477,102,8600.0
Antigua & Barbuda ,LATIN AMER. & CARIB    ,69108,443,11000.0
Argentina ,LATIN AMER. & CARIB    ,39921833,2766890,11200.0
Armenia ,C.W. OF IND. STATES ,2976372,29800,3500.0
Aruba ,LATIN AMER. & CARIB    ,71891,193,28000.0
Australia ,OCEANIA                            ,20264082,7686850,29000.0
Austria ,WESTERN EUROPE                     ,8192880,83870,30000.0
Azerbaijan ,C.W. OF IND. STATES ,7961619,86600,3400.0
Bahamas The ,LATIN AMER. & CARIB    ,303770,13940,16700.0
Bahrain ,NEAR EAS

In [114]:
! cat countries.csv | cut -d',' -f1,3,5  # разделяем данные по запятой и берем только 6 и 7 столбец

Country,Population,GDP ($ per capita)
Afghanistan ,31056997,700.0
Albania ,3581655,4500.0
Algeria ,32930091,6000.0
American Samoa ,57794,8000.0
Andorra ,71201,19000.0
Angola ,12127071,1900.0
Anguilla ,13477,8600.0
Antigua & Barbuda ,69108,11000.0
Argentina ,39921833,11200.0
Armenia ,2976372,3500.0
Aruba ,71891,28000.0
Australia ,20264082,29000.0
Austria ,8192880,30000.0
Azerbaijan ,7961619,3400.0
Bahamas The ,303770,16700.0
Bahrain ,698585,16900.0
Bangladesh ,147365352,1900.0
Barbados ,279912,15700.0
Belarus ,10293011,6100.0
Belgium ,10379067,29100.0
Belize ,287730,4900.0
Benin ,7862944,1100.0
Bermuda ,65773,36000.0
Bhutan ,2279723,1300.0
Bolivia ,8989046,2400.0
Bosnia & Herzegovina ,4498976,6100.0
Botswana ,1639833,9000.0
Brazil ,188078227,7600.0
British Virgin Is. ,23098,16000.0
Brunei ,379444,18600.0
Bulgaria ,7385367,7600.0
Burkina Faso ,13902972,1100.0
Burma ,47382633,1800.0
Burundi ,8090068,600.0
Cambodia ,13881427,1900.0
Cameroon ,17340702,1800.0
Canada ,33098932,29800.0
Cape Ve

In [115]:
# удаляем заголовок и смотрим только на Population
! cat countries.csv | tail -n +2 | cut -d',' -f3 | head

31056997
3581655
32930091
57794
71201
12127071
13477
69108
39921833
2976372


In [116]:
# Считаем уникальные оценки в графе Region
! cat countries.csv | tail -n +2 | cut -d',' -f2 | sort | uniq | wc -l

11


In [117]:
! uname -a  # информация о системе

Linux rc1a-dataproc-m-ymnek55ntqs3neie.mdb.yandexcloud.net 5.4.0-132-generic #148-Ubuntu SMP Mon Oct 17 16:02:06 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux


In [118]:
! uname -a | cut -d" " -f1,3,12  # получаем информацию конкретно про ядро

Linux 5.4.0-132-generic x86_64


### Grep

grep позволяет фильтровать входной поток по указанному регулярному выражению

In [119]:
! head countries.csv

Country,Region,Population,Area (sq. mi.),GDP ($ per capita)
Afghanistan ,ASIA (EX. NEAR EAST)         ,31056997,647500,700.0
Albania ,EASTERN EUROPE                     ,3581655,28748,4500.0
Algeria ,NORTHERN AFRICA                    ,32930091,2381740,6000.0
American Samoa ,OCEANIA                            ,57794,199,8000.0
Andorra ,WESTERN EUROPE                     ,71201,468,19000.0
Angola ,SUB-SAHARAN AFRICA                 ,12127071,1246700,1900.0
Anguilla ,LATIN AMER. & CARIB    ,13477,102,8600.0
Antigua & Barbuda ,LATIN AMER. & CARIB    ,69108,443,11000.0
Argentina ,LATIN AMER. & CARIB    ,39921833,2766890,11200.0


In [120]:
! cat countries.csv | grep "NORTHERN AFRICA"  # Ищем только NORTHERN AFRICA

Algeria ,NORTHERN AFRICA                    ,32930091,2381740,6000.0
Egypt ,NORTHERN AFRICA                    ,78887007,1001450,4000.0
Libya ,NORTHERN AFRICA                    ,5900754,1759540,6400.0
Morocco ,NORTHERN AFRICA                    ,33241259,446550,4000.0
Tunisia ,NORTHERN AFRICA                    ,10175014,163610,6900.0
Western Sahara ,NORTHERN AFRICA                    ,273008,266000,


In [121]:
! cat countries.csv | grep "Islands"

Cayman Islands ,LATIN AMER. & CARIB    ,45436,262,35000.0
Cook Islands ,OCEANIA                            ,21388,240,5000.0
Faroe Islands ,WESTERN EUROPE                     ,47246,1399,22000.0
Marshall Islands ,OCEANIA                            ,60422,11854,1600.0
N. Mariana Islands ,OCEANIA                            ,82459,477,12500.0
Solomon Islands ,OCEANIA                            ,552438,28450,1700.0
Virgin Islands ,LATIN AMER. & CARIB    ,108605,1910,17200.0


### AWK

Программа для обработки структурированного потока со своим собственным небольшим языком программирования.

In [122]:
! awk 'BEGIN{print "Hello World!"; exit}' 

Hello World!


In [123]:
! awk '{ if (length($0) > 70) print $0 }' countries.csv

Australia ,OCEANIA                            ,20264082,7686850,29000.0
Bosnia & Herzegovina ,EASTERN EUROPE                     ,4498976,51129,6100.0
Burkina Faso ,SUB-SAHARAN AFRICA                 ,13902972,274200,1100.0
Central African Rep. ,SUB-SAHARAN AFRICA                 ,4303356,622984,1100.0
Congo Dem. Rep. ,SUB-SAHARAN AFRICA                 ,62660551,2345410,700.0
Congo Repub. of the ,SUB-SAHARAN AFRICA                 ,3702314,342000,700.0
Cote d'Ivoire ,SUB-SAHARAN AFRICA                 ,17654843,322460,1400.0
Czech Republic ,EASTERN EUROPE                     ,10235455,78866,15700.0
Equatorial Guinea ,SUB-SAHARAN AFRICA                 ,540109,28051,2700.0
French Polynesia ,OCEANIA                            ,274578,4167,17500.0
Marshall Islands ,OCEANIA                            ,60422,11854,1600.0
Micronesia Fed. St. ,OCEANIA                            ,108004,702,2000.0
Netherlands ,WESTERN EUROPE                     ,16491461,41526,28600.0
New Caledonia ,OCEANIA  

In [125]:
! cat  countries.csv |  awk -F, '{ if (length($1) > 15) print $2 }'

LATIN AMER. & CARIB    
EASTERN EUROPE                     
LATIN AMER. & CARIB    
SUB-SAHARAN AFRICA                 
SUB-SAHARAN AFRICA                 
SUB-SAHARAN AFRICA                 
LATIN AMER. & CARIB    
SUB-SAHARAN AFRICA                 
OCEANIA                            
OCEANIA                            
OCEANIA                            
LATIN AMER. & CARIB    
OCEANIA                            
OCEANIA                            
LATIN AMER. & CARIB    
NORTHERN AMERICA                   
LATIN AMER. & CARIB    
SUB-SAHARAN AFRICA                 
OCEANIA                            
LATIN AMER. & CARIB    
LATIN AMER. & CARIB    
NEAR EAST                          
OCEANIA                            


In [126]:
! awk -F, '{ if (FNR%15==0) print $0 }' countries.csv

Azerbaijan ,C.W. OF IND. STATES ,7961619,86600,3400.0
British Virgin Is. ,LATIN AMER. & CARIB    ,23098,153,16000.0
Colombia ,LATIN AMER. & CARIB    ,43593035,1138910,6300.0
East Timor ,ASIA (EX. NEAR EAST)         ,1062777,15007,500.0
Gambia The ,SUB-SAHARAN AFRICA                 ,1641564,11300,1700.0
Guyana ,LATIN AMER. & CARIB    ,767245,214970,4000.0
Japan ,ASIA (EX. NEAR EAST)         ,127463611,377835,28200.0
Libya ,NORTHERN AFRICA                    ,5900754,1759540,6400.0
Mauritius ,SUB-SAHARAN AFRICA                 ,1240827,2040,11400.0
New Caledonia ,OCEANIA                            ,219246,19060,15000.0
Poland ,EASTERN EUROPE                     ,38536869,312685,11100.0
Sao Tome & Principe ,SUB-SAHARAN AFRICA                 ,193413,1001,1200.0
Suriname ,LATIN AMER. & CARIB    ,439117,163270,4000.0
Turks & Caicos Is ,LATIN AMER. & CARIB    ,21152,430,9600.0
Western Sahara ,NORTHERN AFRICA                    ,273008,266000,


In [127]:
! cat countries.csv | tail -n +2 | awk -F, 'BEGIN{sum=0.0} {sum+=$3} END{print sum}'

6.52404e+09


In [128]:
! cat countries.csv | tail -n +2 | awk -F, 'BEGIN{max=0.0; country=""} {if ($4 > max) {max = $4; country=$1}} END{print max,country}' 

17075200 Russia 


In [129]:
! cat countries.csv | grep Russia

Russia ,C.W. OF IND. STATES ,142893540,17075200,8900.0


### Jq
jq не является стандартной программой и ее необходимо самостоятельно установить. Для Ubuntu - apt-get install jq.

jq - это манипулятор JSON документами. Имеет свой язык запросов к JSON, чем то похожий на пайплайны в bash.

Подробнее узнать можно здесь - https://stedolan.github.io/jq/

В файле covid.json содержатся записи о заболеваниях короновирусом в различных странах.

In [130]:
! wget https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json

--2023-01-17 16:28:33--  https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81998 (80K) [text/plain]
Saving to: ‘pokedex.json’

pokedex.json        100%[===================>]  80.08K  --.-KB/s    in 0.03s   

2023-01-17 16:28:33 (3.03 MB/s) - ‘pokedex.json’ saved [81998/81998]



In [131]:
! jq --help | head

jq - commandline JSON processor [version 1.6]

Usage:	jq [options] <jq filter> [file...]
	jq [options] --args <jq filter> [strings...]
	jq [options] --jsonargs <jq filter> [JSON_TEXTS...]

jq is a tool for processing JSON inputs, applying the given filter to
its JSON text inputs and producing the filter's results as JSON on
standard output.



In [132]:
! head pokedex.json

{
  "pokemon": [{
    "id": 1,
    "num": "001",
    "name": "Bulbasaur",
    "img": "http://www.serebii.net/pokemongo/pokemon/001.png",
    "type": [
      "Grass",
      "Poison"
    ],


Основной формат запроса в jq - это json path - путь из ключей, по которому нужно пройтись. Каждый такой path генерирует новый поток данных, согласно этому запросу, который можно дальше модифицировать.

Самый короткий запрос это . - то есть мы запрашиваем весть документ целиком. Попробуем посмотреть на первый элемент в массиве measures.

In [133]:
! cat pokedex.json | jq '.pokemon[0]'

{
  "id": 1,
  "num": "001",
  "name": "Bulbasaur",
  "img": "http://www.serebii.net/pokemongo/pokemon/001.png",
  "type": [
    "Grass",
    "Poison"
  ],
  "height": "0.71 m",
  "weight": "6.9 kg",
  "candy": "Bulbasaur Candy",
  "candy_count": 25,
  "egg": "2 km",
  "spawn_chance": 0.69,
  "avg_spawns": 69,
  "spawn_time": "20:00",
  "multipliers": [
    1.58
  ],
  "weaknesses": [
    "Fire",
    "Ice",
    "Flying",
    "Psychic"
  ],
  "next_evolution": [
    {
      "num": "002",
      "name": "Ivysaur"
    },
    {
      "num": "003",
      "name": "Venusaur"
    }
  ]
}


In [134]:
! cat pokedex.json | jq '.pokemon[0].weaknesses'

[
  "Fire",
  "Ice",
  "Flying",
  "Psychic"
]


In [135]:
! cat pokedex.json | jq '. | keys'

[
  "pokemon"
]


In [136]:
! cat pokedex.json | jq '.pokemon[0] | keys'

[
  "avg_spawns",
  "candy",
  "candy_count",
  "egg",
  "height",
  "id",
  "img",
  "multipliers",
  "name",
  "next_evolution",
  "num",
  "spawn_chance",
  "spawn_time",
  "type",
  "weaknesses",
  "weight"
]


In [137]:
! cat pokedex.json | jq '.pokemon[0].next_evolution[0] | keys'

[
  "name",
  "num"
]


In [138]:
! cat pokedex.json | jq '.pokemon[3:5]'

[
  {
    "id": 4,
    "num": "004",
    "name": "Charmander",
    "img": "http://www.serebii.net/pokemongo/pokemon/004.png",
    "type": [
      "Fire"
    ],
    "height": "0.61 m",
    "weight": "8.5 kg",
    "candy": "Charmander Candy",
    "candy_count": 25,
    "egg": "2 km",
    "spawn_chance": 0.253,
    "avg_spawns": 25.3,
    "spawn_time": "08:45",
    "multipliers": [
      1.65
    ],
    "weaknesses": [
      "Water",
      "Ground",
      "Rock"
    ],
    "next_evolution": [
      {
        "num": "005",
        "name": "Charmeleon"
      },
      {
        "num": "006",
        "name": "Charizard"
      }
    ]
  },
  {
    "id": 5,
    "num": "005",
    "name": "Charmeleon",
    "img": "http://www.serebii.net/pokemongo/pokemon/005.png",
    "type": [
      "Fire"
    ],
    "height": "1.09 m",
    "weight": "19.0 kg",
    "candy": "Charmander Candy",
    "candy_count": 100,
    "egg": "Not in Eggs",
    "spawn_chance": 0.012,
    "avg_spawns": 1.2,
    "spawn_time": "1

In [139]:
! cat pokedex.json | jq '.pokemon | length'  # В массиве 151 элемент

151


In [140]:
! cat pokedex.json | jq '.pokemon[0:100] | .[].spawn_chance'

0.69
0.042
0.017
0.253
0.012
0.0031
0.58
0.034
0.0067
3.032
0.187
0.022
7.12
0.44
0.051
15.98
1.02
0.13
13.05
0.41
4.73
0.15
2.27
0.072
0.21
0.0076
1.11
0.037
1.38
0.088
0.012
1.31
0.083
0.017
0.92
0.012
0.22
0.0077
0.39
0.018
6.52
0.42
1.02
0.064
0.0097
2.36
0.074
2.28
0.072
0.4
0.014
0.86
0.022
2.54
0.087
0.92
0.031
0.92
0.017
2.19
0.13
0.011
0.42
0.027
0.0073
0.49
0.034
0.0068
1.15
0.072
0.0059
0.81
0.082
1.19
0.071
0.0047
0.51
0.011
1.05
0.036
0.71
0.023
0.0212
0.52
0.22
0.28
0.013
0.052
0.0031
0.52
0.015
0.79
0.052
0.0067
0.1
3.21
0.1
2.12
0.062
0.65


In [141]:
! cat pokedex.json | jq '.pokemon[].candy' | sort | uniq

"Abra Candy"
"Bellsprout Candy"
"Bulbasaur Candy"
"Caterpie Candy"
"Charmander Candy"
"Clefairy Candy"
"Cubone Candy"
"Diglett Candy"
"Doduo Candy"
"Dratini Candy"
"Drowzee Candy"
"Dugtrio"
"Eevee Candy"
"Ekans Candy"
"Exeggcute Candy"
"Gastly Candy"
"Geodude Candy"
"Goldeen Candy"
"Grimer Candy"
"Growlithe Candy"
"Horsea Candy"
"Jigglypuff Candy"
"Kabuto Candy"
"Koffing Candy"
"Krabby Candy"
"Machop Candy"
"Magikarp Candy"
"Magnemite Candy"
"Mankey Candy"
"Meowth Candy"
"Nidoran ♀ (Female) Candy"
"Nidoran ♂ (Male) Candy"
"None"
"Oddish Candy"
"Omanyte Candy"
"Paras Candy"
"Pidgey Candy"
"Pikachu Candy"
"Poliwag Candy"
"Ponyta Candy"
"Psyduck Candy"
"Rattata Candy"
"Rhyhorn Candy"
"Sandshrew Candy"
"Seel Candy"
"Shellder Candy"
"Slowpoke Candy"
"Spearow Candy"
"Squirtle Candy"
"Staryu Candy"
"Tentacool Candy"
"Venonat Candy"
"Voltorb Candy"
"Vulpix Candy"
"Weedle Candy"
"Zubat Candy"


In [142]:
! cat pokedex.json | jq -r '.pokemon[].candy' | sort | uniq

Abra Candy
Bellsprout Candy
Bulbasaur Candy
Caterpie Candy
Charmander Candy
Clefairy Candy
Cubone Candy
Diglett Candy
Doduo Candy
Dratini Candy
Drowzee Candy
Dugtrio
Eevee Candy
Ekans Candy
Exeggcute Candy
Gastly Candy
Geodude Candy
Goldeen Candy
Grimer Candy
Growlithe Candy
Horsea Candy
Jigglypuff Candy
Kabuto Candy
Koffing Candy
Krabby Candy
Machop Candy
Magikarp Candy
Magnemite Candy
Mankey Candy
Meowth Candy
Nidoran ♀ (Female) Candy
Nidoran ♂ (Male) Candy
None
Oddish Candy
Omanyte Candy
Paras Candy
Pidgey Candy
Pikachu Candy
Poliwag Candy
Ponyta Candy
Psyduck Candy
Rattata Candy
Rhyhorn Candy
Sandshrew Candy
Seel Candy
Shellder Candy
Slowpoke Candy
Spearow Candy
Squirtle Candy
Staryu Candy
Tentacool Candy
Venonat Candy
Voltorb Candy
Vulpix Candy
Weedle Candy
Zubat Candy


С помощью jq можно фильтровать запросы. Для этого есть оператор select

In [143]:
! cat pokedex.json | jq '.pokemon[] | select(.type[0] == "Water")' | head -n 30

{
  "id": 7,
  "num": "007",
  "name": "Squirtle",
  "img": "http://www.serebii.net/pokemongo/pokemon/007.png",
  "type": [
    "Water"
  ],
  "height": "0.51 m",
  "weight": "9.0 kg",
  "candy": "Squirtle Candy",
  "candy_count": 25,
  "egg": "2 km",
  "spawn_chance": 0.58,
  "avg_spawns": 58,
  "spawn_time": "04:25",
  "multipliers": [
    2.1
  ],
  "weaknesses": [
    "Electric",
    "Grass"
  ],
  "next_evolution": [
    {
      "num": "008",
      "name": "Wartortle"
    },
    {
      "num": "009",
Error: writing output failed: Broken pipe


In [144]:
! cat pokedex.json | jq '.pokemon[] | select(.weaknesses[] | contains("Grass")) | .name'

"Squirtle"
"Wartortle"
"Blastoise"
"Sandshrew"
"Sandslash"
"Diglett"
"Dugtrio"
"Psyduck"
"Golduck"
"Poliwag"
"Poliwhirl"
"Poliwrath"
"Geodude"
"Graveler"
"Golem"
"Slowpoke"
"Slowbro"
"Seel"
"Dewgong"
"Shellder"
"Cloyster"
"Onix"
"Krabby"
"Kingler"
"Cubone"
"Marowak"
"Rhyhorn"
"Rhydon"
"Horsea"
"Seadra"
"Goldeen"
"Seaking"
"Staryu"
"Starmie"
"Magikarp"
"Lapras"
"Vaporeon"
"Omanyte"
"Omastar"
"Kabuto"
"Kabutops"


### Archives
Очень часто данные хранятся в виде архивов. Команды tar и zip\unzip позволяют распаковывать архивы.

tar имеет целый набор однобуквенных ключей, комбинация которых позволяет производить различные операции над архивами.

`c` - создать архив

`x` - распаковать архив

`z` - использовать алгоритм gzip. Архивы, созданные с таким алгоритмом, имеют расширение .tar.gz

`v` - печатать на экран детали распаковки

`f` - считать архив из указанного файла

zip\unzip работает немного проще. Команда zip создает новый архив, команда unzip распаковывает указанный архив.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=tar&category=1 и здесь - https://www.opennet.ru/man.shtml?topic=unzip&category=1&russian=4

In [145]:
# Создадим архив из файла pokedex
! tar -czvf pokedex.tar.gz pokedex.json

pokedex.json


In [146]:
! head -n 2 pokedex.tar.gz

�      ��n�6��s��0r��@C?�Uw�t:3��L�t�	�@�ն�咷Jǽ`�o���O�T�>��9�x��� ����E���x��CuW\����V��	�K���%Q����~r2Ƥ�Y$c��3Fⳋ�X:|��:__\|��ܮ���s������/.�xP"��V_,.~n�W?)��7������^}���/����E󣯶��|�o��?��7��o��a�嗏����X�e�����sn��/U��������i���w��o��f�������T2�b��/�����m��_���������q�]��q���������ݏ��׫J����x������a�����C�����6_]5��8k�%����ݿn��������ܿ:.°k�~��ˇeY�����Q�K�w�w�b�)6�/�u�.��򗫃o^/����6/m�tu[^=k������j���Á?m>�э�_~{ڍ����zqa��0�]����/���^h\ׅ�}0Yf|F�e$3&���3�υ�}U_���onn6��B�!�ɀ��\q�κ_m��������a]���!3%5؝VS�&)T�
"HIC_g[Vy$ i�(���t!#te�#�E�)�����w�����?VWw�Kid�b���e��bPO1�{�|}�+*����8YQUQ,�4���8I�VP��YAYf۳%���c�$�����_i��u�Ő����;��� �`��4���AJ�����BHXj�R�7��^�U�R����h*�5��z���r]/������]#pYDlX����(����T[�c+"��l����E�����O���vC��'�b�|S�p��'�Փ��>�,�t�|`'�s���m�$�$�CrDI,�3(i�%JLI���ɣ	,���!,#Lmt�5.�|��Š����ŰM�\�N�$�$��d�bX�I���d�n����+��6�|�E�Dj#�������*��gB�!p'.B������������iN������k\J _mk�r߯��t6�6�}6YJ

In [147]:
# Распакуем этот архив в новую директорию tar-grades
! mkdir -p tar-pokemon && tar -xzvf pokedex.tar.gz -C tar-pokemon/

pokedex.json


In [148]:
! ls tar-pokemon/

pokedex.json


In [149]:
! head tar-pokemon/pokedex.json

{
  "pokemon": [{
    "id": 1,
    "num": "001",
    "name": "Bulbasaur",
    "img": "http://www.serebii.net/pokemongo/pokemon/001.png",
    "type": [
      "Grass",
      "Poison"
    ],


In [150]:
# Сделаем точно тоже самое, но с помощью zip\unzip
! zip pokedex.zip pokedex.json

  adding: pokedex.json (deflated 90%)


In [151]:
! head -n 2 pokedex.zip

�Pؒ��N�)Y��L�*S��l���$*�ZB�2/��hQ�H�"H��X(B���r�G��~�&H+2��������y)'�+r�k�FIs�@-eA���?�l۪�����f�(&w�$v[���m����N?�e�����4�u�Q�lW1�S�?W}(6���bSQ1T���ي���A�7��q�����AE�:�����c�"BYs�;����K�����
�1�h*55���z��]��.K��R�����c�-6�}��6�8Cr�~k��BR;���l��9�E�7����O�f��O!-�Ŗ��V��KZ=����TO��l�d䃝H)�	���"$$DXIڇ���D͠�1�(�%���-�'��Xn


In [159]:
! mkdir -p zip-pokemon && unzip pokedex.zip -d zip-pokemon/

Archive:  pokedex.zip
replace zip-pokemon/pokedex.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [161]:
! ls zip-pokemon/

pokedex.json


In [162]:
! head zip-pokemon/pokedex.json

{
  "pokemon": [{
    "id": 1,
    "num": "001",
    "name": "Bulbasaur",
    "img": "http://www.serebii.net/pokemongo/pokemon/001.png",
    "type": [
      "Grass",
      "Poison"
    ],


Также достаточно популярен формат архивов `7z`. Чтобы начать работать с ним, необходимо поставить дополнительно специальный пакет - `apt-get install p7zip-full`.


In [164]:
! sudo apt-get update && sudo apt-get install p7zip-full -y

Hit:1 http://mirror.yandex.ru/ubuntu focal InRelease
Hit:2 http://storage.yandexcloud.net/dataproc/ci/trunk/80-d30331cee3848e39 focal InRelease
Get:3 http://mirror.yandex.ru/ubuntu focal-updates InRelease [114 kB]          
Get:4 http://mirror.yandex.ru/ubuntu focal-backports InRelease [108 kB]        
Get:5 http://mirror.yandex.ru/mirrors/postgresql focal-pgdg InRelease [91.6 kB]
Hit:6 https://repo.saltproject.io/py3/ubuntu/20.04/amd64/3002 focal InRelease  
Hit:7 https://repos.influxdata.com/ubuntu focal InRelease                      
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://mirror.yandex.ru/ubuntu focal-updates/main amd64 Packages [2,326 kB]
Get:10 http://mirror.yandex.ru/ubuntu focal-updates/main i386 Packages [772 kB]
Get:11 http://mirror.yandex.ru/ubuntu focal-updates/main Translation-en [402 kB]
Get:12 http://mirror.yandex.ru/ubuntu focal-updates/main amd64 c-n-f Metadata [16.1 kB]
Get:13 http://mirror.yandex.ru/ubuntu focal-updates

In [165]:
! 7z | head


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Usage: 7z <command> [<switches>...] <archive_name> [<file_names>...]
       [<@listfiles...>]

<Commands>
  a : Add files to archive
  b : Benchmark


In [166]:
! 7z a pokedex.7z pokedex.json  # Создаем архив


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Scanning the drive:
  0M Sca        1 file, 81998 bytes (81 KiB)

Creating archive: pokedex.7z

Items to compress: 1

    
Files read from disk: 1
Archive size: 7044 bytes (7 KiB)
Everything is Ok


In [167]:
! head -n 2 pokedex.7z

]v�7:�^z8ӺV$��Ds΂�6i=Ke$z-�]��^G<@�_	k������]��>�q��T�^�%U����5��ϔd���4ܝ]&�`���7$b�g���x��U)�&�r�u��
;�K$�j�4)�=���nд3������?#d��4�G���#��4	�Q�H*��_��h���c�1L#�4>�


In [168]:
! mkdir -p 7z-pokedex && 7z x pokedex.7z -o7z-pokedex/  #  Распаковываем


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 7044 bytes (7 KiB)

Extracting archive: pokedex.7z
--
Path = pokedex.7z
Type = 7z
Physical Size = 7044
Headers Size = 130
Method = LZMA2:96k
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       81998
Compressed: 7044


In [169]:
! ls 7z-pokedex/

pokedex.json


### Networking
wget и curl позволяют выгружать данные из интернета.

wget более продвинутый - он умеет скачивать сразу множество файлов, поддерживает докачку файлов и так далее. curl более простой и может использоваться скорее для точечных запросов.

Подробнее можно узнать здесь - https://www.opennet.ru/man.shtml?topic=wget&category=1&russian=0 и здесь - https://www.opennet.ru/man.shtml?topic=curl&category=1&russian=3



In [170]:
! mkdir -p nets

In [171]:
%cd nets

/home/ubuntu/tempsem2/nets


In [172]:
! curl -L https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json > pokedex.json.curl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 81998  100 81998    0     0   454k      0 --:--:-- --:--:-- --:--:--  454k


In [173]:
! wget https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json -O pokedex.json.wget

--2023-01-17 16:49:08--  https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81998 (80K) [text/plain]
Saving to: ‘pokedex.json.wget’

pokedex.json.wget   100%[===================>]  80.08K  --.-KB/s    in 0.03s   

2023-01-17 16:49:08 (2.88 MB/s) - ‘pokedex.json.wget’ saved [81998/81998]



In [174]:
! wget https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json -O - | head

--2023-01-17 16:49:35--  https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81998 (80K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               {
  "pokemon": [{
    "id": 1,
    "num": "001",
    "name": "Bulbasaur",
    "img": "http://www.serebii.net/pokemongo/pokemon/001.png",
    "type": [
      "Grass",
      "Poison"
    ],
-                    11%[=>                  ]   9.42K  --.-KB/s    in 0.001s  


Cannot write to ‘-’ (Success).


In [175]:
! ls

pokedex.json.curl  pokedex.json.wget


Создатим список ссылок и скачаем их все разом с помощью ключа -i



In [176]:
%%writefile link-list.txt
https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json
https://raw.githubusercontent.com/ADKosm/lsml-2022-public/main/data/2/countries.csv

Writing link-list.txt


In [177]:
! wget -i link-list.txt

--2023-01-17 16:50:20--  https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81998 (80K) [text/plain]
Saving to: ‘pokedex.json’

pokedex.json        100%[===================>]  80.08K  --.-KB/s    in 0.02s   

2023-01-17 16:50:20 (3.19 MB/s) - ‘pokedex.json’ saved [81998/81998]

--2023-01-17 16:50:20--  https://raw.githubusercontent.com/ADKosm/lsml-2022-public/main/data/2/countries.csv
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 14641 (14K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]  14.30K  --.-KB/s    in 0s      

2023-01-17 16:50:20 (53.8 MB/s) - ‘countries.csv’ save

In [178]:
! ls

countries.csv  pokedex.json	  pokedex.json.wget
link-list.txt  pokedex.json.curl


In [181]:
! hdfs dfs -mkdir -p /user/pokemons

In [182]:
! curl https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json | hdfs dfs -put - /user/pokemons/pokedex.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 81998  100 81998    0     0  51152      0  0:00:01  0:00:01 --:--:-- 51120


In [183]:
! hdfs dfs -ls /user/pokemons

Found 1 items
-rw-r--r--   1 ubuntu hadoop      81998 2023-01-17 16:52 /user/pokemons/pokedex.json


### HDFS

Подключимся к мастер-ноде кластера и попробуем с него поработать с HDFS

```bash
ssh lsml-head

hdfs dfs -ls s3a://lsml-kosmos/
wget https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json

sudo apt-get update && sudo apt-get install jq -y
cat pokedex.json | jq -r '.pokemon[] | select(.weaknesses[] | contains("Water")) | .name' > water_weak.txt

hdfs dfs -put water_weak.txt s3a://lsml-kosmos/water_weak.txt

hdfs dfs -ls s3a://lsml-kosmos/
hdfs dfs -get s3a://lsml-kosmos/water_weak.txt hdfs_water_weak.txt
```